# chapter 5 - Recommendation Systems
추천 시스템
1. 협업 필터링 (CF) Collaborative Filtering 
2. 콘텐츠 기반 필터링 (CBF) Content-based Filtering

기타 접근방법
1. 연관 룰 association rules
2. $Log$ 우도 the log-likelihood method
3. 하이브리드 hybrid methods

<br></br>
## 1 유틸리티 행렬
Utility matrix 추천시스템에 사용되는 계량척도 시스템

평점 사용자 $i$가 아이템 $j$를 평가한 사용자 평가목록 : $ r_{ij}$ 

### 01 데이터 수집 및 전처리
import Csv files 

In [14]:
import numpy as np
import pandas as pd
from time import time
from scipy import linalg
from collections import defaultdict
import copy, collections, math

# 사용자 평점 데이터 불러오기
df = pd.read_csv('./data/ml-100k/u.data', sep='\t', header=None)
movies_rated  = list(df[1]) 
counts = collections.Counter(movies_rated)
df.head(2)

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742


In [15]:
# https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
# 데이터 영화목록 불러오기
df_info = pd.read_csv('./data/ml-100k/u.item', 
                      sep='|', header=None, encoding = "ISO-8859-1")
# 영화목록 추출 (index 라인 번호를 제목 뒤에 덧붙임)
movie_list = [df_info[1].tolist()[ index_num ] + ';' + str(index_num+1) 
              for index_num in range(len(df_info[1].tolist())) ]
df_info.head(2)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
df_out = pd.DataFrame(columns=['user']+movie_list)
print(df_out.shape)
df_out.ix[:,:6]  # 영화의 목록을 컬럼으로 생성

(0, 1683)


,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5


In [ ]:
min_ratings = 50  # 작성자 리뷰중 50개 이하만 기록된 영화 데이터는 제거
num_users  = len(df[0].drop_duplicates().tolist()) # 전체 사용자 수  
num_movies = len(movie_list) # 전체 영화 수


In [21]:
to_removelist = []; t0 = time()
for i in range(1, num_users):
    tmp_movielist = [0 for j in range(num_movies)]
    df_tmp = df[df[0] == i]

    for k in df_tmp.index:
        if counts[ df_tmp.ix[k][1] ] >= min_ratings:
            tmp_movielist[ df_tmp.ix[k][1] -1 ] = df_tmp.ix[k][2]
        else: to_removelist.append(df_tmp.ix[k][1])
    df_out.loc[i] = [i] + tmp_movielist   # df의 사용자 리뷰 데이터를 df_out 에 첨부

to_remove_list = list(set(to_removelist))
df_out.drop(df_out.columns[ to_removelist ], axis = 1, inplace = True)
df_out.to_csv('data/utilitymatrix.csv', index = None)
print(round(time()-t0,4),'sec'); print(df_out.shape); df_out.iloc[:3,:7]

31.0997 sec
(942, 604)


,user,Toy Story (1995);1,GoldenEye (1995);2,Four Rooms (1995);3,Get Shorty (1995);4,Copycat (1995);5,Twelve Monkeys (1995);7
1,1.0,5.0,3.0,4.0,3.0,3.0,4.0
2,2.0,4.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# 01 내용 정리
# 총 1683개의 영화에 대한 개별 사용자의 평점을 첨부한다
# 50개 이하의 평점이 기록된 영화는 제거되어, 최종 604개의 영화기록이 정제된다

### 02 결측치 처리
NaN 결측치는, '대체'(imputation)의 방법으로 초기화 한다

대체 (imputation) : <strong>사용자의 평균 평점</strong>과 <strong>영화별 평균 평점</strong>을 대체에 사용한다.

In [12]:
def imputation(inp,Ri):
    Ri = Ri.astype(float)
    def userav():
        for i in range(len(Ri)):
            Ri[i][Ri[i] == 0] = sum( Ri[i] ) / float(len(Ri[i][Ri[i] > 0]))
        return Ri
    def itemav():
        for i in range(len(Ri[0])):
            Ri[:, i][Ri[:, i] == 0] = sum(Ri[:,i])/float(len(Ri[:, i][Ri[:, i] > 0]))
        return Ri            
    switch = {'useraverage' : userav() ,'itemaverage' : itemav() }
    return switch[inp]

In [23]:
# 유틸리티 행렬 R은
# 사용자는 N명이고, 아이템은 M개가 된다

<br></br>
## 2 유사도 척도
벡터 $X$와 $Y$의 유사도를 계산하기 위한 척도
1. 코사인 유사도 Cosine similarity 
$$ s(x,y) = \frac{\sqrt{\sum x_i y_i}}{\sqrt{\sum x_i^2}\sqrt{\sum y_i^2}} $$
2. 피어슨 상관관계 Pearson correlation
$$ s(x,y) = \frac{\sqrt{\sum (x_i - \bar x)(y_i - \bar y)}}
{\sqrt{\sum (x_i - \bar x)^2}\sqrt{\sum (y_i - \bar y)^2}} $$

두 척도의 평균이 0인경우, 일치하게 된다

In [24]:
# scipy를 활용한 유사도 척도 함수
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def similarity(x,y,metric='cos'):
    if metric == 'cos':
        return 1.-cosine(x,y)   # 코사인 유사도를 출력
    else:
        return pearsonr(x,y)[0] # 피어슨 상관계수를 출력

<br></br>
## 3 협업 필터링 방법
<h4><strong>Collaborative Filtering methods</strong></h4>
"자신과 비슷한 사람들은, 좋아하는 아이템을 공유한다"를 기반으로 한다

이는 사용자의 수많은 데이터를 필요로 해서 발생하는 <strong>Cold Start</strong>

(반대로 <strong>가용 데이터 부족</strong>의 <strong>Warm Start</strong>가 있다)가 문제가 되고 있다

이를 극복하기 위하여, <strong>CF</strong>와 <strong>CBF의 하이브리드</strong> 방식이 제안된다

(콘텐츠의 내부적 분류로써, 사용자의 데이터를 더 세분화 한 데이터로 분석을 한다)
<h4><strong>콘텐츠 기반 알고리즘의 이슈</strong></h4>
공통이슈로 <strong>'확장성(Scalability)'</strong>이 언급되는데

이는 사용자와 아이템 갯수에 비례하여 연산량이 급증하는 문제를 말한다(<strong>병렬처리</strong>가 필요)

반면 아이템 갯수가 작은경우에는 <strong>희소성(Sparsity)</strong> 문제가 생긴다(<strong>대체</strong>의 방법으로 해결


<h4>1. <strong>메모리 기반의 협업 필터링</strong> Memory-based Collaborative Filtering</h4> 
<strong>유틸리티 행렬</strong>을 사용하여 아이템의 유사도를 계산한다

<h4>2. <strong>사용자 기반 협업 필터링</strong> User-based Collaborative Filtering</h4>
<strong> K-NN 알고리즘</strong>으로, 유사 사용자들의 평점을 찾은 뒤 

<strong>가중 평균</strong>으로,누락 평점 대신에 사용된다
1. 사용자 $i$와 <strong>평가되지 않은 아이템 $j$</strong>를 특정
2. <strong>유사도 척도 $s$</strong>를 사용, $j$아이템에 유사평점을 준 사용자 $K$를 찾는다 ($20 < K < 50$)
3. $K$들의 평점을 <strong>가중평균</strong>하여, 사용자 $i$의 평점을 예측한다
4. 균질적인 평점을 비교하기 위하여, <strong>평점분포를 정규화</strong> 한다
5. 예측평균이 1~5 기본 척도값 이외가 나온경우, 최소값은 1, 최대값은 5로 조정

In [15]:
def CF_userbased(u_vec, K, data, indxs = False):
    def FindKNeighbours(r, data, K):
        neighs, cnt = [], 0
        for u in range(len(data)):
            if data[u,r] > 0  and  cnt < K:
                neighs.append(data[u])
                cnt += 1 
            elif cnt == K: break
        return np.array(neighs)

    def CalcRating(u_vec, r, neighs):
        rating, den = 0., 0.
        for j in range(len(neighs)):
            rating += neighs[j][-1] * float(neighs[j][r] - neighs[j][neighs[j] > 0][:-1].mean())
            den += abs(neighs[j][-1])
        if den > 0:    rating = np.round(u_vec[u_vec>0].mean()+(rating/den),0)
        else:          rating = np.round(u_vec[u_vec>0].mean(),0)
        if rating > 5: return 5.
        elif rating<1: return 1.
        return rating 

    data = data.astype(float)
    nrows, ncols = len(data), len(data[0])
    data_sim = np.zeros((nrows,ncols+1))
    data_sim[:,:-1] = data

    # calc similarities:
    for u in range(nrows):
        if np.array_equal(data_sim[u, :-1], u_vec) == False: 
            data_sim[u,ncols] = sim(data_sim[u,:-1],u_vec,'pearson')
        else:
            data_sim[u,ncols] = 0.

    #order by similarity:
    data_sim =data_sim[data_sim[:,ncols].argsort()][::-1]
    #find the K users for each item not rated:
    u_rec = np.zeros(len(u_vec))
    for r in xrange(ncols):
        if u_vec[r]==0:
            neighs = FindKNeighbours(r,data_sim,K)
            # calc the predicted ratin
            u_rec[r] = CalcRating(u_vec,r,neighs)

    #take out the rated movies
    if indxs:
        seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
        u_rec[seenindxs] = -1
        recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]
        return recsvec    
    return u_rec

In [ ]:
# 위 함수 사용해본 결과
# 피어슨 상관관계가 조금 더 결과가 좋은데, 사용자의 평균평점을 제외한채 사용하기 때문으로 보인다

<h4>2. <strong>아이템 기반의 협업 필터링</strong> Item-based Collaborative Filtering</h4> 
<strong>아이템에 대해 유사도가 측정</strong>된다는 점을 제외하고는 개념적으로, 사용자기반과 동일하다
1. <strong>유사도 측정치 $s$</strong>를 사용하여, 사용자 $i$가 평가한 아이템과 가장 <strong>비슷한 아이템을 $K$개</strong> 찾는다
2. $K$개 아이템 평점을 가중평균하여 예측평점으로 계산한다
$$ P_{ij} = \frac{\sum S(j,k)r_{ik}}{|\sum{S(j,k)}|} $$

유사도 측정결과 음수도 가능하므로, <strong>가중 평균의 유효성을 높이기 위해서</strong> 양수값들만의 가중평균을 사용하면 더 효과적이다

In [ ]:
class CF_itembased(object):
    def __init__(self,data):
        #calc item similarities matrix
        nitems = len(data[0])
        self.data = data
        self.simmatrix = np.zeros((nitems,nitems))
        for i in xrange(nitems):
            for j in xrange(nitems):
                if j>=i: # tri-angular matrix
                    self.simmatrix[i,j] = sim(data[:,i],data[:,j])
                else: self.simmatrix[i,j] = self.simmatrix[j,i]

    def GetKSimItemsperUser(self,r,K,u_vec):
        items = np.argsort(self.simmatrix[r])[::-1]
        items = items[items!=r]
        cnt=0
        neighitems = []
        for i in items:
            if u_vec[i]>0 and cnt<K:
                neighitems.append(i)
                cnt+=1
            elif cnt==K: break
        return neighitems
        
    def CalcRating(self,r,u_vec,neighitems):
        rating = 0.
        den = 0.
        for i in neighitems:
            rating +=  self.simmatrix[r,i]*u_vec[i]
            den += abs(self.simmatrix[r,i])
        if den>0:
            rating = np.round(rating/den,0)
        else:
            rating = np.round(self.data[:,r][self.data[:,r]>0].mean(),0)
        return rating
        
    def CalcRatings(self,u_vec,K,indxs=False):
        #u_rec = copy.copy(u_vec)
        u_rec = np.zeros(len(u_vec))
        for r in xrange(len(u_vec)):
            if u_vec[r]==0:
                neighitems = self.GetKSimItemsperUser(r,K,u_vec)
                #calc predicted rating
                u_rec[r] = self.CalcRating(r,u_vec,neighitems)
        if indxs:
            #take out the rated movies
            seenindxs = [indx for indx in xrange(len(u_vec)) if u_vec[indx]>0]
            u_rec[seenindxs]=-1
            recsvec = np.argsort(u_rec)[::-1][np.argsort(u_rec)>0]        
            return recsvec
        return u_rec

In [ ]:
# 클래스 내용 정리
# simmatrix로 아이템 유사도 행렬을 계산한다
# 아이템의 사용자 평점이 없는 경우, CalcRatings로 '가중평균 평점'을 예측을 한다
# GetKSimItemsperUsers는 '사용자가 평가하지 않은 아이템과 유사도가 높은 아이템' 중
# 사용자가 과거에 평가했던 K개의 아이템을 찾는다
# 이웃을 찾을 수 없는 경우는, 아이템의 평균평점으로 설정한다
# u_vec 는 사용자 평점 벡터로, 유틸리티 행렬의 행 벡터다